ChatBot - LLM chatbot - have conversations - remembers previous interactions

Conversational RAG -> enables a chatbot experience over a external source of data

Agents -> Build a chatbot that does actions

In [1]:
import langchain
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020BCA4A2800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020BCA4A33D0>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
result = model.invoke([HumanMessage(content = "Hi, my name is Harish and I am a student")])
result

AIMessage(content="Hello Harish, it's nice to meet you!\n\nI'm glad you introduced yourself.  \n\nWhat can I do for you today? Are you working on a school project, need help with something, or just want to chat?  😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 21, 'total_tokens': 76, 'completion_time': 0.1, 'prompt_time': 0.00013669, 'queue_time': 0.020749787000000002, 'total_time': 0.10013669}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc56b16c-1d37-4f64-b3d4-a8b698ae2ced-0', usage_metadata={'input_tokens': 21, 'output_tokens': 55, 'total_tokens': 76})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content = "Hi, my name is Harish and I am a student"),
    AIMessage(content="Hi Harish, it's nice to meet you!  \n\nWhat can I help you with today? Are you working on a project, studying for a test, or just looking to chat?  \n\nI'm here to assist in any way I can. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 21, 'total_tokens': 79, 'completion_time': 0.105454545, 'prompt_time': 0.000140969, 'queue_time': 0.019906550999999998, 'total_time': 0.105595514}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-13ef5b17-7096-48bf-9ca0-266ab57dbea4-0', usage_metadata={'input_tokens': 21, 'output_tokens': 58, 'total_tokens': 79}),
   HumanMessage(content = "What am I?")
])
# chatbots can remember the previous messages

AIMessage(content="That's a fun question!  \n\nSince you told me you're a student, that's what you are!  You're a learner, someone who is gaining knowledge and skills. \n\nIs there anything else you'd like to explore about yourself or the world around you? I'm ready for more questions! 😄 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 91, 'total_tokens': 165, 'completion_time': 0.134545455, 'prompt_time': 0.003443509, 'queue_time': 0.020153338, 'total_time': 0.137988964}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5302503c-8afc-420a-8e80-d43373bf4b72-0', usage_metadata={'input_tokens': 91, 'output_tokens': 74, 'total_tokens': 165})

### Message History
we can make the message history class to wrap our model and make it stateful. This will keep track of our inputs and outputs and store it in some DB.

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory # very important

store = {}

# session id distinguishes the different sessions
def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# using this RunnableWithMessageHistory to interact with LLM model based on the chat History
with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [6]:
config = {"configurable" : {"session_id" : "chat_1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hi my name is Harish, I am a student")],
    config= config
    )
response.content

"Hello Harish,\n\nIt's nice to meet you!\n\nWhat are you studying?  Are you working on any interesting projects right now?\n"

In [8]:
## change the config
config = {"configurable" : {"session_id" : "chat_1"}}
response = with_message_history.invoke(
    [HumanMessage(content = "what am I?")],
    config= config
    )
response.content

"That's a fun question!  \n\nSince you told me your name is Harish and that you are a student, I can say that you are:\n\n* **A person:**  You have a name and you identify as a student.\n* **A learner:**  Being a student means you are actively seeking knowledge and understanding.\n* **A future something:**  What will you become after you finish your studies? A doctor? An engineer? An artist? The possibilities are endless!\n\n\nIs there anything else you want to tell me about yourself? I'm curious to learn more!\n"

### Prompt templates

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [("system","you are helpful assistant. answer all the questions to the best of your ability"),
     MessagesPlaceholder(variable_name = "messages")] # whatever human message we give it should be key value pairs, thats why we have variable_names as messages which is a key
)

chain = prompt | model

In [10]:
chain.invoke({"messages" : [HumanMessage(content = "My name is harish")]})

AIMessage(content="Hello Harish! It's nice to meet you. \n\nI'm ready to answer your questions to the best of my ability. Just ask away! 😊  \n\nWhat can I help you with today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 29, 'total_tokens': 78, 'completion_time': 0.089090909, 'prompt_time': 0.000135569, 'queue_time': 0.019914329, 'total_time': 0.089226478}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-808d458d-6d7a-4955-acd8-2a184cc6e7f5-0', usage_metadata={'input_tokens': 29, 'output_tokens': 49, 'total_tokens': 78})

In [11]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [12]:
config = {"configurable" : {"session_id" : "chat_3"}}

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hi my name is Harish, I am a student and I like Bananas")],
    config= config
    )
response.content

"Hi Harish, it's nice to meet you!\n\nI'm glad to hear you like bananas. They're a great source of potassium and energy. \n\nWhat can I help you with today?  Do you have any questions about bananas, or something else entirely? 😊 \n\n"

In [14]:
## change the config
config = {"configurable" : {"session_id" : "chat_3"}}
response = with_message_history.invoke(
    [HumanMessage(content = "what am I and which fruit I like?")],
    config= config
    )
response.content

"You are a student, Harish, and you like bananas!  🍌  \n\nIs there anything else you'd like to know or talk about?  \n"

In [15]:
# increasing the complexity of the prompt
prompt = ChatPromptTemplate.from_messages(

    [
        ("system","you are helpful assistant. answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name = "messages")
     ] # whatever human message we give it should be key value pairs, thats why we have variable_names as messages which is a key

)

chain = prompt | model

In [16]:
response = chain.invoke({"messages" : [HumanMessage(content = "Hello How are you")], "language" : "spanish"})
response

AIMessage(content='¡Hola! Estoy muy bien, gracias. ¿Y tú?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 30, 'total_tokens': 47, 'completion_time': 0.030909091, 'prompt_time': 0.000166379, 'queue_time': 0.019697789, 'total_time': 0.03107547}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2079a16d-c6fe-4258-8be5-77f95b967080-0', usage_metadata={'input_tokens': 30, 'output_tokens': 17, 'total_tokens': 47})

In [17]:
# increasing the complexity of the prompt
# 
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = "messages"
)

In [18]:
config = {"configurable" : {"session_id" : "chat_4"}}

In [19]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content = "hello how are you?")],
        "language" : "japanese"
    },
    config = config
)
response.content

'こんにちは！元気ですか？😊 \n(Konnichiwa! Genki desu ka?)\n\nThis translates to "Hello! How are you?" in Japanese. \n\n\nLet me know if you have any other questions!  \n'

### Managing the conversation history

In [43]:
# trim_messages helps to reduce how many messages we re sending to the model
# it allows us to specify how many tokens we want to keep, along with other parameters
from langchain_core.messages import SystemMessage, trim_messages
trim_message = trim_messages(
    max_tokens = 50,
    strategy = "last",
    token_counter = model, 
    include_system = True, # including the system message
    allow_partial = False ,# no need of partial infos
    start_on = "human"
    )

In [44]:
messages = [
    SystemMessage(content = "You re a very good assistant"),
    HumanMessage(content = "I am harish and I like vanilla Icecream"),
    AIMessage(content = "that sounds delicious"),
    HumanMessage(content = "Hello"),
    AIMessage(content = "Hi! how may I help you?"),
    HumanMessage(content = "Nothing.. help me with my project"),
    AIMessage(content = "sure"),
    HumanMessage(content = "Its about implementing the chatbot using langchain"),
    AIMessage(content = "Oh thats great! In which phase can I help you?")

]

trim_message.invoke(messages)

[SystemMessage(content='You re a very good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Nothing.. help me with my project', additional_kwargs={}, response_metadata={}),
 AIMessage(content='sure', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Its about implementing the chatbot using langchain', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Oh thats great! In which phase can I help you?', additional_kwargs={}, response_metadata={})]

In [45]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trim_message) |
    prompt |
    model 
)
chain.invoke(
    {"messages" : messages + [HumanMessage(content = "What Project am I doing?")],
     "language" : "english"}
    
)

AIMessage(content="You're working on a project to implement a chatbot using LangChain!  \n\nTo give you the best help, tell me more about what you're trying to achieve. For example:\n\n* **What kind of chatbot are you building?** (e.g., customer service, educational, creative writing assistant)\n* **What data will your chatbot use?** (e.g., a specific dataset, access to the internet)\n* **What specific challenges are you facing?** (e.g., choosing the right model, structuring your data, integrating with a platform)\n\n\nThe more details you provide, the more tailored and helpful my assistance can be. 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 69, 'total_tokens': 212, 'completion_time': 0.26, 'prompt_time': 0.002037684, 'queue_time': 0.019923362, 'total_time': 0.262037684}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f14aa9e6-b7e4-4459-8fb3-0508d

In [46]:
chain.invoke(
    {"messages" : messages + [HumanMessage(content = "What Icecream do I like?")],
     "language" : "english"}
    
)

AIMessage(content="As a helpful assistant, I don't have access to your personal preferences like your favorite ice cream flavor. \n\nTo figure out what ice cream you like, maybe think about:\n\n* **Your favorite fruits:** Do you like berry flavors, chocolate, or something tropical?\n* **Your favorite desserts:** Do you prefer rich and creamy flavors, or something lighter and tangy?\n* **Your favorite candies:**  Do you like peanut butter, caramel, or cookie dough?\n\n\n\nLet me know if you want to brainstorm some ideas based on your likes!🍦😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 70, 'total_tokens': 191, 'completion_time': 0.22, 'prompt_time': 0.002368223, 'queue_time': 0.027426883, 'total_time': 0.222368223}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a15ac46a-4f95-4d30-bee4-67c1e8dbcf38-0', usage_metadata={'input_tokens': 70, 'output_tokens

In [48]:
# wrapping this in the message history
config = {"configurable" : {"session_id" : "chat_5"}}
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = "messages"
)

In [50]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content = "hello how are you?")],
        "language" : "English"
    },
    config = config
)
response.content

"Hello! I'm doing well, thank you.  \n\nHow can I help you today? 😊  \n\n"

In [52]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content = "I like icecreams")],
        "language" : "english"
    },
    config = config
)
response.content

"That's great! Ice cream is delicious. \n\nWhat's your favorite flavor?  🍦😊  \n\n"

In [53]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content = "help me?")],
        "language" : "english"
    },
    config = config
)
response.content

"I'd love to!  \n\nTo help me understand how to best assist you, could you please tell me what you need help with? \n\nFor example, are you:\n\n* **Looking for information** on a specific topic?\n* **Trying to solve a problem**?\n* **Needing creative ideas** for something?\n* **Just wanting to chat**?\n\n\nThe more details you give me, the better I can help! 😊  \n\n"

In [54]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content = "what do I like?")],
        "language" : "english"
    },
    config = config
)
response.content

"As an AI, I have no access to your past interactions, preferences, or personal information.  Therefore, I can't know what you like! \n\nTo help me understand your interests, tell me:\n\n* **What are your hobbies?**\n* **What kind of books, movies, or music do you enjoy?**\n* **What do you like to do in your free time?**\n* **Is there anything you're passionate about?**\n\n\nThe more you tell me, the better I can get to know you and your likes! 😊 \n\n"